In [2]:
import os

import numpy as np

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

import pandas as pd


In [3]:
from torchvision.io import read_image
from torchvision.models import vgg16, VGG16_Weights, resnet152, ResNet152_Weights
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
from sklearn.decomposition import PCA
import numpy as np
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from scipy.spatial import distance
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd


# Заменю последний слой этим, чтобы получать эмбеддинги, а не классы.
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# weights = VGG16_Weights.DEFAULT
# model = vgg16(weights=weights)
# model.classifier = nn.Sequential(*[model.classifier[i] for i in range(4)])

weights = ResNet152_Weights.DEFAULT
model = resnet152(weights=weights)
model.fc = Identity()


device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
from PIL import Image

myimage = Image.open("data/merged/images/by-brands/under armour/333.85066")  
myimage.load()



In [10]:
myimage.format

'JPEG'

In [6]:
for p in model.parameters():
    p.requires_grad = False

In [11]:
preprocess = weights.transforms()
dataset = ImageFolder("data/merged/images/by-brands", transform=preprocess)
dataloader = DataLoader(
    dataset, batch_size=64, shuffle=False, drop_last=False, num_workers=4
)


In [12]:
dataset.imgs

[('data/merged/images/by-brands/361 degrees/0.png', 0),
 ('data/merged/images/by-brands/361 degrees/1.png', 0),
 ('data/merged/images/by-brands/361 degrees/2.png', 0),
 ('data/merged/images/by-brands/361 degrees/3.jpeg', 0),
 ('data/merged/images/by-brands/361 degrees/4.jpeg', 0),
 ('data/merged/images/by-brands/361 degrees/5.jpeg', 0),
 ('data/merged/images/by-brands/a cold wall/0.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/1.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/10.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/11.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/12.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/13.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/14.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/15.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/16.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/17.jpg', 1),
 ('data/merged/images/by-brands/a cold wall/18.jpg', 1),
 ('data/merged/images/by-brands/a co

In [13]:
next(iter(dataloader))

[tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
           [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
           [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
           ...,
           [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
           [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
           [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],
 
          [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
           [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
           [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
           ...,
           [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
           [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
           [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],
 
          [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
           [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
           [2.6400, 2.64